# GMM para Speech features

## Importando libs

In [1]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import multivariate_normal
import math
from numpy import linalg as LA
import matplotlib.cm as cm
import matplotlib.mlab as mlab

%matplotlib inline

## Inicializando variáveis globais

In [2]:
Ng = 3
test_size = .2

## Lendo os dados MFCC do arquivo

In [3]:
target_frame_sanderson = pd.read_csv('data/feature_lib.csv')
target_frame_sanderson.fillna(0, inplace=True)
y = target_frame_sanderson.iloc[:, 1].values
X = target_frame_sanderson.iloc[:, 2:100].values
y, X

(array([ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,
         1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  0.]),
 array([[ 16.06516658,  -6.167928  , -11.60263281, ...,   9.73565104,
          -6.01127765,  -0.39201773],
        [ 15.52645668,  -5.77143651,  -8.6210234 , ...,  25.84498242,
          -4.89838865,  11.15632049],
        [ 15.90670822,  -0.70229411, -11.62260717, ...,   1.67244094,
          -9.22338872,  -0.79974016],
        ..., 
        [ 16.03676337,  -3.01418019,  -2.49913762, ...,  14.47599253,
          -1.42811305,   2.77662196],
        [ 19.67184283, -14.94721738, -16.43764792, ...,   8.47230223,
          -9.10652143,   2.46218451],
        [ 16.03121297,  -3.3894565 , -12.06090029, ...,  14.06559439,
          -4.03672792,  -6.78197443]]))

## Separando treino e teste para validação cruzada

In [4]:
data, tar = X, y

def get_key(t):
    return t[1]

res = list(zip(data, tar))
res = sorted(res, key=get_key)
data, tar = list(zip(*res))
d0 = data[0:10]
d1 = data[10:20]
d2 = data[20:30]

t0 = tar[0:10]
t1 = tar[10:20]
t2 = tar[20:30]

dataset, target = np.array([d0, d1, d2]), np.array([t0, t1, t2])

n_interaction = [round(len(d) * test_size) for d in dataset]

data_tes = []
target_tes = []
data_train = []
target_train = []
for d in range(0, len(dataset)):
    for i in range(0, n_interaction[d]):
        rand_i = random.randint(0, len(dataset[d]) - 1)
        data_tes.append(dataset[d][rand_i])
        target_tes.append(target[d][rand_i])
        new_dataset = np.delete(dataset[d], rand_i, axis=0)
        new_target = np.delete(target[d], rand_i, axis=0)
    data_train.append(new_dataset)
    target_train.append(new_target)
X_train, y_train, X_test, y_test = data_train, target_train, data_tes, target_tes

## Dividindo o treinamento em classes

In [5]:
c0, c1, c2 = X_train

c0 = np.array(c0)
c1 = np.array(c1)
c2 = np.array(c2)

data = c0

## Gerando modelo para uma classe

###  Usando Kmeans para melhor inicialização

#### Categorizar cada vetor com a média mais próxima

In [6]:
def __min_norm_gauss(mu, Xi):
    norms = [LA.norm(mu[g] - Xi) for g in range(0, Ng)]
    return norms.index(min(norms))

def __min_index_gauss(mu, data):
    return [__min_norm_gauss(mu, data[i]) for i in range(0, len(data))]

#### Calculando médias (centros das gaussianas)

In [7]:
max_interation = 10

def __means(X):
    def __new_mu(data, delta_kronecker):
        gf = [delta_kronecker(i) for i in range(0, len(data))]
        if sum(gf) == 0:
            return 0, np.zeros(len(data[0]))
        mu_updated = (1 / sum(gf)) * sum([data[ix] * y for ix, y in enumerate(gf)])
        return sum(gf), mu_updated

    mu = [random.choice(X) for i in range(0, Ng)]

    loop = 1
    finished = False
    yi = []
    ng = []
    while not finished:
        yi = __min_index_gauss(mu, X)
        res = [__new_mu(X, lambda ix: yi[ix] == g) for g in range(0, Ng)]
        ng, mu_new = zip(*res)
        ng = list(ng)
        mu_new = list(mu_new)

        same = all([np.array_equal(mu[g], mu_new[g]) for g in range(0, Ng)])
        loop = loop + 1
        finished = same or loop > max_interation

        mu = mu_new
    return ng, mu, yi
    
__means(data)

([1, 5, 3], [array([  1.55264567e+01,  -5.77143651e+00,  -8.62102340e+00,
           4.52386983e+00,   1.23436361e+01,  -6.48672500e-01,
           3.74219567e+00,   2.48026113e+00,  -4.88078293e-01,
          -1.10245653e+00,  -5.83638335e-01,   5.69465550e-02,
           2.53794473e-03,   1.56510099e+01,  -5.42779910e+00,
          -9.82064204e+00,   3.94246045e+00,   1.58950272e+01,
          -1.04415332e+00,   2.85519568e+00,   1.96449013e+00,
           4.91899807e-01,  -1.63378161e+00,   8.56987046e-02,
           9.41299791e-01,   1.99346133e-01,   1.56507798e+01,
          -5.75622502e+00,  -1.06653390e+01,   2.31122390e+00,
           1.47847773e+01,   1.84206721e+00,   9.24126926e-01,
          -1.12439053e+00,  -2.05334889e+00,  -2.33678228e+00,
           1.55861799e-01,   7.91353791e-01,   3.99784032e-02,
           1.56786722e+01,  -4.38488403e+00,  -9.20446116e+00,
           4.14061663e+00,   1.62258305e+01,   4.34324133e+00,
           3.64715819e+00,   1.46091882e+00,

#### Calcula o peso e matriz de covariancia

In [8]:
def kmeans(data):
    ng, mu, y = __means(data)
    mg = [ng[g] / len(data) for g in range(0, Ng)]
    sigmas = [np.cov((data).T) for g in range(0, Ng)]
    sigmas = np.array(sigmas) 
    return mg, mu, sigmas

kmeans(data)

([0.3333333333333333, 0.4444444444444444, 0.2222222222222222],
 [array([ 15.89159688,  -3.17582068,  -5.2348719 ,   6.81459002,
          12.04649419,   2.54307378,   2.04799039,   3.6903804 ,
           2.04608831,   0.27116337,  -0.05670477,   0.1111363 ,
          -0.10158427,  15.69830384,  -2.38411886,  -6.29684887,
           6.54013522,  16.72973633,   0.82450364,   2.06058399,
           2.06812168,   2.59272263,   0.27087342,   0.31840982,
           0.56589164,  -0.11848016,  15.66953332,  -1.98312026,
          -6.0635499 ,   5.71940148,  16.38171463,   2.94043643,
           1.99376331,   0.95881417,   1.22396536,   0.41550301,
           0.34936579,   0.54096586,  -0.11570069,  15.74870757,
          -1.55768158,  -4.7150152 ,   7.55437005,  16.99680645,
           3.73447994,   4.08970612,   3.14610763,   2.05179853,
           1.02877061,   1.1236739 ,   0.79558916,  -0.07919179,
          15.6877426 ,  -1.3686988 ,  -5.22119288,   6.47539715,
          15.61951967,   4.

### Funçoes para obter PDF da gaussiana dada lambda

In [16]:
    def gauss(x, u, e):
        e += 0.3 * np.identity(len(e))
        res = multivariate_normal(u, e)
        return res.pdf(x)

    def weighted_gauss(x, m, u, e):
        return m * gauss(x, u, e)

    def sum_weighted_gauss(x, lamb):
        acc = []
        for g in range(0, Ng):
            m_arr = lamb[0]
            u_arr = lamb[1]
            e_arr = lamb[2]
            acc.append(weighted_gauss(x, m_arr[g], u_arr[g], e_arr[g]))
        return sum(acc)

    def get_prob(data, lamb):
        acc = []
        for i in range(0, len(data)):
            acc.append(sum_weighted_gauss(data[i], lamb))
        return sum(acc)

    def get_prob_log( data, lamb):
        acc = []
        for i in range(0, len(data)):
            acc.append(math.log(sum_weighted_gauss(data[i], lamb)))
        return sum(acc)

## Maximizar lambda

In [17]:
def calc_lg(data, lamb, g):
    for i in range(0, len(data)):
        aux1 = weighted_gauss(data[i], lamb[0][g], lamb[1][g], lamb[2][g])
        aux2 = sum_weighted_gauss(data[i], lamb)
        yield aux1 / aux2

def __expectation_maximization(X, lam):

    def ug_concat(x, lg_aux): return [x[i] * lg_aux[i] for i in range(0, len(x))]

    def safe_div(x, y):
        if y == 0:
            return 0
        return x / y

    lgi = [list(calc_lg(X, lam, g)) for g in range(0, Ng)]
    lg = [sum(lgi[g]) for g in range(0, Ng)]

    weight = [lg[g] / len(X) for g in range(0, Ng)]
    mu = [(safe_div(1, lg[g]) * sum(ug_concat(X, lgi[g]))) for g in range(0, Ng)]
    sigma = [np.cov((X - mu[g]).T) for g in range(0, Ng)]

    return weight, mu, sigma

([0.0075385903574132461, 20.766938337212817, 0.0],
 [array([  1.55264569e+01,  -5.77143325e+00,  -8.62102532e+00,
           4.52386360e+00,   1.23436298e+01,  -6.48676241e-01,
           3.74219567e+00,   2.48026089e+00,  -4.88076847e-01,
          -1.10245596e+00,  -5.83638091e-01,   5.69473345e-02,
           2.53815165e-03,   1.56510103e+01,  -5.42779665e+00,
          -9.82064323e+00,   3.94245615e+00,   1.58950189e+01,
          -1.04415822e+00,   2.85519760e+00,   1.96448864e+00,
           4.91900230e-01,  -1.63378044e+00,   8.56976834e-02,
           9.41299932e-01,   1.99345948e-01,   1.56507804e+01,
          -5.75622218e+00,  -1.06653403e+01,   2.31122095e+00,
           1.47847719e+01,   1.84205925e+00,   9.24128063e-01,
          -1.12439133e+00,  -2.05334548e+00,  -2.33677972e+00,
           1.55860350e-01,   7.91353698e-01,   3.99782875e-02,
           1.56786726e+01,  -4.38488217e+00,  -9.20446389e+00,
           4.14061179e+00,   1.62258249e+01,   4.34323350e+00,
    

## Máxima verossimilhança (EM)

In [19]:
def __maximum_likelihood( X, lamb):
    for i in range(0, max_interation):
        p_old = get_prob_log(X, lamb)
        lamb_new = __expectation_maximization(X, lamb)
        p_new = get_prob_log(X, lamb_new)
        if p_new > p_old:
            lamb = lamb_new
    return lamb

([0.0025715707744842944, 0.0, 2.753104169925217],
 [array([  1.55264784e+01,  -5.77113225e+00,  -8.62115447e+00,
           4.52335589e+00,   1.23430612e+01,  -6.48989152e-01,
           3.74217274e+00,   2.48023282e+00,  -4.87954086e-01,
          -1.10240640e+00,  -5.83617198e-01,   5.70149550e-02,
           2.55512224e-03,   1.56510526e+01,  -5.42758804e+00,
          -9.82077381e+00,   3.94210330e+00,   1.58943402e+01,
          -1.04461627e+00,   2.85538729e+00,   1.96432619e+00,
           4.91918754e-01,  -1.63366491e+00,   8.56262355e-02,
           9.41315335e-01,   1.99326257e-01,   1.56508424e+01,
          -5.75600433e+00,  -1.06655156e+01,   2.31099815e+00,
           1.47843838e+01,   1.84127548e+00,   9.24295067e-01,
          -1.12451523e+00,  -2.05304866e+00,  -2.33652608e+00,
           1.55730428e-01,   7.91347609e-01,   3.99651727e-02,
           1.56787272e+01,  -4.38476022e+00,  -9.20476743e+00,
           4.14022816e+00,   1.62254297e+01,   4.34245903e+00,
     

## Criar uma função para gerar os modelos

In [21]:
def model(data):
    lamb = kmeans(data)
    return __maximum_likelihood(data, lamb)

model(data)

([5.421759001508653e-12, 0.05395541192157377, 2.7528944116110319],
 [array([  1.59051149e+01,  -7.23319283e-01,  -1.16093538e+01,
          -5.14497702e+00,   2.43155548e+00,  -6.44859408e+00,
           3.74078069e+00,   2.12206299e+00,   1.75501092e+00,
          -2.27709615e-01,  -2.05191535e-01,   1.26644397e+00,
           3.23654240e-01,   1.62705212e+01,  -1.63557550e+00,
          -1.16786426e+01,  -2.71492119e+00,   3.05965590e+00,
          -8.64828514e+00,   5.83146849e+00,  -3.41112543e-01,
           1.14925168e+00,   1.92624878e-01,  -1.49922386e+00,
           1.16014511e+00,  -8.75859987e-02,   1.64787220e+01,
          -1.34987686e+00,  -1.26826401e+01,  -2.27625187e+00,
           6.44867444e+00,  -1.05100228e+01,   2.69106157e+00,
          -2.36535850e+00,   3.24840751e+00,   1.64226683e+00,
          -2.09207422e+00,   6.47041048e-01,  -1.39433088e-01,
           1.63655049e+01,  -1.50710010e+00,  -1.34329366e+01,
          -3.36169757e+00,   7.57208723e+00,  -7.79

## Criar um modelo para cada classe

In [23]:
models = [model(c0), model(c1), model(c2)]
models

[([0.11935977605794428, 0.0, 2.7526493039219408],
  [array([  1.55264809e+01,  -5.77109725e+00,  -8.62116955e+00,
            4.52329677e+00,   1.23429951e+01,  -6.49025579e-01,
            3.74217010e+00,   2.48022956e+00,  -4.87939797e-01,
           -1.10240064e+00,  -5.83614767e-01,   5.70228235e-02,
            2.55709829e-03,   1.56510575e+01,  -5.42756376e+00,
           -9.82078897e+00,   3.94206222e+00,   1.58942611e+01,
           -1.04466953e+00,   2.85540933e+00,   1.96430733e+00,
            4.91920934e-01,  -1.63365148e+00,   8.56178986e-02,
            9.41317123e-01,   1.99323971e-01,   1.56508496e+01,
           -5.75597894e+00,  -1.06655359e+01,   2.31097218e+00,
            1.47843386e+01,   1.84118441e+00,   9.24314408e-01,
           -1.12452957e+00,  -2.05301412e+00,  -2.33649661e+00,
            1.55715316e-01,   7.91346898e-01,   3.99636508e-02,
            1.56787335e+01,  -4.38474597e+00,  -9.20480266e+00,
            4.14018347e+00,   1.62253836e+01,   4.3423

## Encontrar melhor classe dado modelos (predict)

In [30]:
def find_best_class(data, models):
    best_p = 0
    best_class = -1
    for c in range(0, len(models)):
        client = models[c]
        p = sum_weighted_gauss(data, client)
        if p > best_p:
            best_p = p
            best_class = c
    return best_class

find_best_class(X_test[0], models)

0

## Utilizar dados para treinamento e comparar com esperado

In [36]:
def compare(data_test, target_test, models):
    accepts = []
    for ix in range(0, len(data_test)):
        data = data_test[ix]
        target = target_test[ix]
        best_class = find_best_class(data, models)

        accepts.append(best_class == target)

    return accepts
compare(X_test, y_test, models)

[True, True, False, False]

## Sumário do resultado

In [38]:
accepts = compare(X_test, y_test, models)
print(accepts)

taxa_accepts = sum(accepts) / len(X_test)
taxa_errors = (len(X_test) - sum(accepts)) / len(X_test)

taxa_accepts, taxa_errors

[True, True, False, False]


(0.5, 0.5)